In [4]:
import h5py
import numpy as np
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [13]:
with h5py.File('SVHN_single_grey1.h5', 'r') as file:
    x_train = np.array(file['X_train'])
    y_train = np.array(file['y_train'])
    x_test = np.array(file['X_test'])
    y_test = np.array(file['y_test'])
    x_val = np.array(file['X_val'])
    y_val = np.array(file['y_val'])

In [14]:
x_train[0].shape

(32, 32)

In [15]:
y_train.shape

(42000,)

In [16]:
# Load and preprocess the SVHN dataset
x_train, x_test = x_train / 255.0, x_test / 255.0

In [17]:
# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

In [18]:
# Convert labels from {0,9} classes into one-hot encoded categorical vectors
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

In [19]:
# Constants
input_shape = x_train.shape[1:]
nClasses = 10

In [20]:
def createModel():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 1)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(10,  activation='softmax'))

    return model

In [21]:
# Create the model
svhn_model = createModel()

In [22]:
# Display the summary of the model
print("SVHN Model Architecture:")
svhn_model.summary()

SVHN Model Architecture:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        320       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (N

In [32]:
AdamOpt = Adam(learning_rate=0.001)
svhn_model.compile(optimizer=AdamOpt,
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

In [33]:
batch_size = 256
epochs = 10

history = svhn_model.fit(x_train,
                         y_train,
                         batch_size=batch_size,
                         epochs=epochs,
                         verbose=1,
                         class_weight=class_weight_dict,
                         validation_data=(x_val, y_val))

Epoch 1/10
165/165 [==============================] - 55s 328ms/step - loss: 2.3672 - accuracy: 0.1006 - val_loss: 2.3034 - val_accuracy: 0.1000
Epoch 2/10
 54/165 [========>.....................] - ETA: 28s - loss: 2.3439 - accuracy: 0.0968

KeyboardInterrupt: 

In [ ]:
# Visualize training and validation errors
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Evaluate the model
test_loss, test_accuracy = svhn_model.evaluate(x_test, y_test)
print(f"\nSVHN Model - Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
# Choose a few images for visualization
sample_images = x_test[:5]
sample_labels = y_test[:5]

In [ ]:
# Visualize predictions
predictions = svhn_model.predict(sample_images)
predicted_labels = [tf.argmax(prediction).numpy() for prediction in predictions]

In [ ]:
# Display the actual and predicted labels
print("\nActual Labels:", sample_labels)
print("Predicted Labels:", predicted_labels)

In [ ]:
# Visualize sample images
for i in range(len(sample_images)):
    plt.figure()
    plt.imshow(sample_images[i], cmap='gray')
    plt.title(f"Actual: {sample_labels[i]}, Predicted: {predicted_labels[i]}")
    plt.show()